# ***2D Molecular dynamics simulation of Aragorn Fluid***

## *importing the packages, Configuration of the Figs and data directories*

In [ ]:
include("Q1-MD.jl")
include("Q1-ExportData.jl")
using Plots, LaTeXStrings, Statistics, JLD, ProgressMeter, Plots.PlotMeasures
using StatsBase: autocor

figpath = "../../Figs/"
datapath = "../../Data/"
animpath = "../../Anims/"
Plots.default(titlefontsize = 12, tickfontsize = 10, labelfontsize = 12, legendfontsize = 9,
    fontfamily = "Computer Modern", frame = :box, label = nothing, rightmargin = 3mm, bottommargin = 2mm)

### *Setting the Parameters and Exporting Data*

In [ ]:
Parameters1 = Dict(:N => 100, :T₀ => 1.0, :h => 0.0015, :l => 50.0)
Parameters2 = Dict(:N => 100, :T₀ => 1.0, :h => 0.0005, :l => 50.0)

StepCount1 = 3000
Runnumver1 = 1000
StepCount2 = 200000


# ExportData.AnsembleData(StepCount1, Runnumver1, Parameters1)
# ExportData.SingleData(StepCount2, Parameters2)

### *Loading and Mining Data*

In [ ]:
TimeCollTD = collect(0:Parameters1[:h]:Parameters1[:h]*StepCount1)[1:StepCount1]
TimeCollSD = collect(0:Parameters2[:h]:Parameters2[:h]*StepCount2)[1:StepCount2]
DataTD = load(datapath * "DataTD.jld")
TotUColl = DataTD["TotUColl"]
TotKColl = DataTD["TotKColl"]
TotTColl = DataTD["TotTColl"]
TotPColl = DataTD["TotPColl"]
TotlsColl = DataTD["TotlsColl"]
DataSD = load(datapath * "DataSD.jld")
PosColl = DataSD["PosColl"]
VelColl = DataSD["VelColl"]
UColl = DataSD["UColl"]
KColl = DataSD["KColl"]
TColl = DataSD["TColl"]
PColl = DataSD["PColl"]
# ExportData.MineData(TotUColl, TotKColl, TotTColl, TotPColl, TotlsColl)
MeanData = load(datapath * "MeanData.jld")
MeanUColl = MeanData["MEANUColl"]
MeanKColl = MeanData["MEANKColl"]
MeanTColl = MeanData["MEANTColl"]
MeanPColl = MeanData["MEANPColl"]
MeanlsColl = MeanData["MEANlsColl"]
STDData = load(datapath * "STDData.jld")
STDUColl = STDData["STDUColl"]
STDKColl = STDData["STDKColl"]
STDTColl = STDData["STDTColl"]
STDPColl = STDData["STDPColl"]
STDlsColl = STDData["STDlsColl"]

## ***Properties of Single System***

### *Equlibrium*

In [ ]:
P1 = begin
    plot(TimeCollSD, [count(<=(Parameters2[:l] / 2), PosColl[i, 1, :]) / Parameters2[:N] for i ∈ 1:StepCount2], c = :steelblue, label = "Particles on left")
    plot!(TimeCollSD, [count(>(Parameters2[:l] / 2), PosColl[i, 1, :]) / Parameters2[:N] for i ∈ 1:StepCount2], c = :purple, label = "Particles on right")
    plot!(title = "Particle density in the right and left halves\n" * L"N_{net}= %$(Parameters2[:N]),\ L_s= %$(Parameters2[:l]),\ \Delta{t}= %$(Parameters2[:h])" * "\n ",
        xlabel = "t/τ", ylabel = L"N_{left}/N_{net}", legend = :topright)
end

savefig(P1, figpath * "Fig1.pdf")

display(P1)

In [ ]:
P2 = begin
    plot(TimeCollSD, UColl, c = :steelblue, label = "Potential Energy")
    plot!(TimeCollSD, KColl, c = :purple, label = "Kinematic Energy")
    plot!(TimeCollSD, UColl + KColl, c = :black, label = "Total Energy", lw = 2.5, legend = 0)
    plot!(xlabel = "t/τ", ylabel = "E/ϵ", title = "Energy of system\n" * L"N_{net}= %$(Parameters2[:N]),\ L_s= %$(Parameters2[:l]),\ \Delta{t}= %$(Parameters2[:h])" * "\n ")
end

savefig(P2, figpath * "Fig2.pdf")

display(P2)

In [ ]:
P3 = begin
    plot(TimeCollSD, 120 * TColl, c = :maroon, label = "Temperature")
    plot!([0.0, 100.0], 120 * [mean(TColl[10000:end]), mean(TColl[10000:end])], c = :black, lw = 3,
        label = L"\langle T_{eq} \rangle = %$(round(120 * mean(TColl[10000:end]), digits = 3))")
    plot!(xlabel = "t/τ", ylabel = "T(K)", title = "Temperature of system\n" * L"N_{net}= %$(Parameters2[:N]),\ L_s= %$(Parameters2[:l]),\ \Delta{t}= %$(Parameters2[:h])" * "\n ")
end

savefig(P3, figpath * "Fig3.pdf")

display(P3)

In [ ]:
P4 = begin
    plot(TimeCollSD, PColl, c = :purple4, label = "Pressure")
    plot!([0.0, 100.0], [mean(PColl[5000:end]), mean(PColl[5000:end])], c = :black, lw = 3,
        label = L"\langle P_{eq} \rangle= %$(round(mean(PColl[5000:end]), digits = 3))")
    plot!(xlabel = "t/τ", ylabel = L"P\sigma^2/\epsilon",
        title = "Pressure of system\n" * L"N_{net}= %$(Parameters2[:N]),\ L_s= %$(Parameters2[:l]),\ \Delta{t}= %$(Parameters2[:h])" * "\n ")
end

savefig(P4, figpath * "Fig4.pdf")

display(P4)

### *Autocorrelation of velocity*

In [ ]:
v = Matrix{Float64}(undef, StepCount2, Parameters2[:N])
for i in 1:StepCount2
    v[i, :] = @. √(VelColl[i, 1, :]^2 + VelColl[i, 2, :]^2)
end

autocorrelation = autocor(v, 1:StepCount2÷5)

P5 = begin
    plot(TimeCollSD[1:StepCount2÷5], mean(autocorrelation, dims = 2), c = :royalblue4, lw = 3)
    plot!(xlabel = "t/τ", ylabel = L"C_v",
        title = "Autocorrelation of particles velocities\n" * L"N_{net}= %$(Parameters2[:N]),\ L_s= %$(Parameters2[:l]),\ \Delta{t}= %$(Parameters2[:h]),\ T_0 = 120K" * "\n ")
end

savefig(P5, figpath * "Fig5.pdf")

display(P5)

In [ ]:
τₑ = findfirst(x -> x <= 0, mean(autocorrelation, dims = 2))[1] - 1
Lt = TimeCollSD[1:τₑ]
logCv = log.(mean(autocorrelation[1:τₑ, :], dims = 2))

_ξ⁻¹, coeff = hcat(Lt, ones(τₑ)) \ logCv
δ_ξ⁻¹ = √(sum((logCv - _ξ⁻¹ * Lt .- coeff) .^ 2) / (τₑ - 2) / sum((lags .- mean(lags)).^2))

ξ = -1 / _ξ⁻¹
δξ = δ_ξ⁻¹ / ξ^2

display(L"ξ = %$ξ,\quad δξ = %$δξ")

## ***Average of Properties of System on thousand runs***

### *Equlibrium*

In [ ]:
P6 = begin
    plot(TimeCollTD[1:60:end], MeanUColl[1:60:end], ribbon = STDUColl[1:60:end],
        ls = :dashdot, lc = :black, c = :steelblue, m = :c, mc = :steelblue, ms = 3, label = "Potential Energy")
    plot!(TimeCollTD[1:60:end], MeanKColl[1:60:end], ribbon = STDKColl[1:60:end],
        ls = :dashdot, lc = :black, c = :purple, m = :c, mc = :purple, ms = 3, label = "Kinematic Energy")
    plot!(TimeCollTD[1:60:end], MeanUColl[1:60:end] + MeanKColl[1:60:end],
        ls = :solid, lc = :black, lw = 2, label = "Total Energy", legend = 0)
    plot!(xlabel = "t/τ", ylabel = L"\langle E_{(t)} \rangle/\epsilon",
        title = "Energy of system\n" * L"N_{net}= %$(Parameters1[:N]),\ L_s= %$(Parameters1[:l]),\ \Delta{t}= %$(Parameters1[:h])" * "\n ")
end

savefig(P6, figpath * "Fig6.pdf")

display(P6)

In [ ]:
P7 = begin
    plot(TimeCollTD, 120 * MeanTColl, ribbon = 120 * STDTColl, c = :maroon, ls = :dashdot, lc = :black, lw = 1.5)
    plot!(xlabel = "t/τ", ylabel = L"\langle T_{(t)} \rangle",
        title = "Average Temperature on 1000 runs\n" * L"N_{net}= %$(Parameters1[:N]),\ L_s= %$(Parameters1[:l]),\ \Delta{t}= %$(Parameters1[:h])" * "\n ")
end

savefig(P7, figpath * "Fig7.pdf")

display(P7)

In [ ]:
P8 = begin
    plot(TimeCollTD, MeanPColl, ribbon = STDPColl, c = :purple4, ls = :dashdot, lc = :black, lw = 1.5)
    plot!(xlabel = "t/τ", ylabel = L"\langle P_{(t)} \rangle\sigma^2/\epsilon",
        title = "Average Pressure on 1000 runs\n" * L"N_{net}= %$(Parameters1[:N]),\ L_s= %$(Parameters1[:l]),\ \Delta{t}= %$(Parameters1[:h])" * "\n ")
end

savefig(P8, figpath * "Fig8.pdf")

display(P8)

## ***Trajectory of System***

### *Argon gas*

In [ ]:
# PosColl = DataSD["PosColl"]



### *Phase Transmission*

In [ ]:
# Parameters = Dict(:N => 100, :T₀ => 1.0, :h => 0.01, :l => 30.0)

# sys = MDSim.init(; Parameters...)

# Temperatures = Float64[]
# Pressures = Float64[]
# anim = Animation()

# rate = [range(1.0, 0.95; length = 5000); fill(0.985, 1000); range(1.0, 0.8; length = 4000)]

# Times = collect(0:Parameters[:h]:Parameters[:h]*length(rate))[1:length(rate)]

# prog = Progress(length(rate))

# t = 0.0
# for i ∈ 1:length(rate)
#     next!(prog)
#     update!(prog)
#     MDSim.update_phase!(sys)
#     MDSim.check_boundary!(sys)
#     MDSim.update_temperature!(sys)
#     MDSim.update_pressure!(sys)
#     push!(Temperatures, sys.T)
#     push!(Pressures, sys.P)
#     if i % 10 == 0
#         sys.v *= rate[i]
#         p = scatter(sys.r[1, :], sys.r[2, :], color = :indigo, msw = 0.01, markersize = 7,
#             title = "\$T = $(round(120 * sys.T, digits=1))\\mathrm{K}," * "\\quad t = $(round(t, digits=2))\\tau\$",
#             xlabel = L"x/\sigma", ylabel = L"y/\sigma", xlim = (1, sys.l - 1), ylim = (1, sys.l - 1), frame = :box,
#             grid = false, tickdir = :none, size = (600, 600), ratio = :equal)
#         frame(anim, p)
#     end
#     t += sys.h
# end

gif(anim, animpath * "PhaseTransmission.gif", fps = 30)


## *Properties of gas during phase transition*

In [ ]:
# plot(Temperatures)
# plot(Pressures)

# display(P9)